In [10]:
import requests
import requests_cache
from tqdm import trange, tqdm
from typing import TypedDict, NamedTuple, Literal, NotRequired

requests_cache.install_cache("tba_cache")

In [12]:
class TeamEventOPR(NamedTuple):
    team: str
    event: str
    opr: float
    week: int
    regional_points: int

class DistrictList(TypedDict):
    abbreviation: str
    display_name: str
    key: str
    year: int

class Webcast(TypedDict):
    type: Literal["youtube", "twitch", "ustream",
                  "iframe", "html5", "rtmp", "livestream", "direct_link",
                  "mms", "justin", "stemtv", "dacast"]
    channel: str
    date: str | None
    file: str | None

class Event(TypedDict):
    key: str
    name: str
    event_code: str
    event_type: int
    district: DistrictList | None
    city: str | None
    state_prov: str | None
    country: str | None
    start_date: str
    end_date: str
    year: int
    short_name: str | None
    event_type_string: str
    week: int
    address: str | None
    postal_code: str | None
    gmaps_place_id: str | None
    gmaps_url: str | None
    lat: float | None
    lng: float | None
    location_name: str | None
    timezone: str
    website: str | None
    first_event_id: str | None
    first_event_code: str | None
    webcasts: list[Webcast]
    division_keys: list[str]
    parent_event_key: str | None
    playoff_type: int | None
    playoff_type_string: str | None

class Team(TypedDict):
    key: str
    team_number: int
    nickname: str
    name: str
    school_name: str | None
    city: str | None
    state_prov: str | None
    country: str | None
    address: str | None
    postal_code: str | None
    gmaps_place_id: str | None
    gmaps_url: str | None
    lat: float | None
    lng: float | None
    location_name: str | None
    website: NotRequired[str | None]
    rookie_year: int | None

class PointsAtEventForTeam(TypedDict):
    total: int
    alliance_points: int
    elim_points: int
    award_points: int
    qual_points: int

class TiebrakersAtEventForTeam(TypedDict):
    highest_qual_scores: list[int]
    qual_wins: list[int]

class DistrictPoints(TypedDict):
    points: dict[str, PointsAtEventForTeam]
    tiebreakers: dict[str, TiebrakersAtEventForTeam]

class OPRs(TypedDict):
    oprs: dict[str, float]
    dprs: dict[str, float]
    ccwms: dict[str, float]

class AwardRecepient(TypedDict):
    team_key: str
    awardee: str

class Award(TypedDict):
    name: str
    award_type: int
    event_key: str
    recepient_list: list[AwardRecepient]
    year: int

In [ ]:
TEAM_PAGES = 22

HEADERS = {
    "X-TBA-Auth-Key": ""
}

ROOT_URL = "https://www.thebluealliance.com/api/v3"

In [ ]:
ALL_TEAM_KEYS: list[str] = []

for i in trange(TEAM_PAGES, desc="Getting all teams", unit="page", leave=False):
    ALL_TEAM_KEYS.extend(
        requests.get(f"{ROOT_URL}/teams/{i}/keys", headers=HEADERS).json()
    )

In [30]:
TEAM_EVENT_AWARDS: dict[str, dict[str, list[Award]]] = {}
EVENT_KEY_MAP: dict[str, str] = {}

for team_key in tqdm(
    ALL_TEAM_KEYS, desc="Getting team events", unit="team", leave=False
):
    TEAM_EVENT_AWARDS[team_key] = {}
    team_events: list[Event] = requests.get(f"{ROOT_URL}/team/{team_key}/events", headers=HEADERS).json()
    for event in team_events:
        if not 0 <= event["event_type"] <= 5:
            continue
        EVENT_KEY_MAP[event["key"]] = (
            event["key"]
            if event["parent_event_key"] is None
            else event["parent_event_key"]
        )
        TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[event["key"]]] = []
    team_awards: list[Award] = requests.get(f"{ROOT_URL}/team/{team_key}/awards", headers=HEADERS).json()
    for award in team_awards:
        if award["event_key"] not in EVENT_KEY_MAP or 3 <= award["award_type"] <= 8:
            continue
        TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[award["event_key"]]].append(award)

KeyError: '2019mrcmp'

In [35]:
team_events

[{'address': 'New Brunswick, NJ 08901, USA',
  'city': 'New Brunswick',
  'country': 'USA',
  'district': None,
  'division_keys': [],
  'end_date': '1998-03-21',
  'event_code': 'nj',
  'event_type': 0,
  'event_type_string': 'Regional',
  'first_event_code': 'nj',
  'first_event_id': '6619',
  'gmaps_place_id': 'ChIJvwa-GFXGw4kRSK96kWYgtFE',
  'gmaps_url': 'https://maps.google.com/?cid=5887366237779570504',
  'key': '1998nj',
  'lat': 40.5008186,
  'lng': -74.44739910000001,
  'location_name': 'Rutgers–New Brunswick',
  'name': 'Johnson & Johnson Mid-Atlantic Regional',
  'parent_event_key': None,
  'playoff_type': None,
  'playoff_type_string': None,
  'postal_code': '08901',
  'short_name': 'Johnson & Johnson Mid-Atlantic',
  'start_date': '1998-03-19',
  'state_prov': 'NJ',
  'timezone': 'America/New_York',
  'webcasts': [],
  'website': None,
  'week': 2,
  'year': 1998},
 {'address': '1801 N Broad St, Philadelphia, PA 19122, USA',
  'city': 'Philadelphia',
  'country': 'USA',
  